# Compare MCTS to other methods

In [ ]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression

n_samples = 1000
n_features = 20
n_classes = 8
n_labels = 2
random_state = 0

X, Y = make_multilabel_classification(
    n_samples = n_samples,
    n_features = n_features,
    n_classes = n_classes,
    n_labels = n_labels,
    random_state = random_state)

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

solver = "liblinear"
base = LogisticRegression(solver=solver)
chain = ClassifierChain(base)

chain = chain.fit(X_train, Y_train)

# Basic loss comparison

In [ ]:
from sklearn.metrics import hamming_loss, zero_one_loss

from mcts_inference.constraints import Constraint
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.mcts import MCTS

secs_lis = [0.1, 0.5, 1.]
M = min(100,len(Y_test))

hl_mt = []
hl_ct = []
hl_mc = []

zo_mt = []
zo_ct = []
zo_mc = []

y_chain = chain.predict(X_test[:M])
for secs in secs_lis:
#     continue  # Comment if you do not want to run this loop.
    y_mcts = []

    config = MCTSConfig(n_classes=n_classes, constraint=Constraint(time=True,d_time=secs), parallel=True, verbose=True)
    y_mcts = MCTS(X_test[:M], chain, config=config)
    
    hl_mt.append(hamming_loss(y_mcts,Y_test[:M]))
    hl_ct.append(hamming_loss(y_chain,Y_test[:M]))
    hl_mc.append(hamming_loss(y_chain,y_mcts))

    zo_mt.append(zero_one_loss(y_mcts,Y_test[:M]))
    zo_ct.append(zero_one_loss(y_chain,Y_test[:M]))
    zo_mc.append(zero_one_loss(y_chain,y_mcts))

# Plot
import matplotlib.pyplot as plt

plt.plot(secs_lis,hl_mt,label="MCTS vs True")
plt.plot(secs_lis,hl_ct,label="Chains vs True")
plt.plot(secs_lis,hl_mc,label="MCTS vs Chains")

plt.title("Hamming Loss Comparison for different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Hamming Loss")
plt.legend()
plt.show();

plt.plot(secs_lis,zo_mt,label="MCTS vs True")
plt.plot(secs_lis,zo_ct,label="Chains vs True")
plt.plot(secs_lis,zo_mc,label="MCTS vs Chains")

plt.title("Zero One Loss Comparison for time different times")
plt.xlabel("Seconds")
plt.ylim(0,1)
plt.ylabel("Zero One Loss")
plt.legend()
plt.show();

## Some averaged runs

In [ ]:
import matplotlib.pyplot as plt
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.pcc import PCC

def avg_losses(Ns, config, chain, algo= MCTS, loss_function = hamming_loss, M: int = 100, avg: int = 10):
    losses = []

    print("> n:",end="")
    for n in Ns:
        print(n, end=", ")
        constraint = Constraint(max_iter=True, n_iter=n)
        config.replace_constraint(constraint)

        avg_loss = 0
        for _ in range(avg):
            out = algo(X_test[:M], chain, config)
            avg_loss += loss_function(out, Y_test[:M])

        losses.append(avg_loss/avg)

    return losses

def plot_losses(Ns, config, algo, chain, bf: bool = True, M: int = 100, label: str = "", title: str = "", loss_function=hamming_loss):
    fig = plt.figure(figsize=(10, 5))
    
    losses = avg_losses(Ns=Ns, config=config, algo=algo, chain=chain, M=M, loss_function=loss_function)
    chain_losses = loss_function(chain.predict(X_test[:M]), Y_test[:M])

    plt.plot(Ns, losses, label=label, marker="o", markersize=3)
    plt.plot(Ns, [chain_losses]*len(Ns), label="Chain", marker="o", markersize=1)

    if bf:
        title += " vs PCC"
        con = MCTSConfig(n_classes, constraint=Constraint(time=True, d_time=2), parallel=True)
        PCC_losses = loss_function(PCC(X_test[:M], chain, config=con), Y_test[:M])
        plt.plot(Ns, [PCC_losses]*len(Ns), label="PCC", marker="o", markersize=1)

    plt.xlabel("Number of Iterations")
    plt.ylabel(loss_function.__qualname__ + " Loss")

    plt.title(title)
    plt.legend()
    plt.show();

In [ ]:
from mcts_inference.mcc import MCC
from mcts_inference.constraints import Constraint
from mcts_inference.mcts_config import MonteCarloConfig

Ns = [n for n in range(5, 206, 20)]
config = MonteCarloConfig(n_classes, constraint=Constraint(max_iter=True, n_iter=2))
plot_losses(Ns, config, algo=MCC, chain=chain, bf = False, label="MCC", title="MCC vs Chain", M=100, loss_function=hamming_loss)